<a href="https://colab.research.google.com/github/antoniodragoc/Projeto-de-Graduacao-Antonio-Drago-Caetano/blob/main/2_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import the needed packages
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
#from keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# https://thedatafrog.com/en/articles/image-recognition-transfer-learning/

from keras.layers.core.flatten import Flatten
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D

## Referências

Artigo principal: https://arxiv.org/pdf/1810.09025.pdf

Configurações: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/ 

Aumento de dados: https://pyimagesearch.com/2019/07/08/=keras-imagedatagenerator-and-data-augmentation/

Aplicação aumento de dados: https://gist.github.com/rstml/bbd491287efc24133b90d4f7f3663905

In [ ]:
# Gerando os dados a partir das imagens
train_datagen = ImageDataGenerator(    
    rotation_range = 90,
    vertical_flip = True,
    horizontal_flip = True,
    zoom_range=0.05,
    width_shift_range=0.2,
		height_shift_range=0.2,
    fill_mode = 'reflect',
    samplewise_center = True,
    samplewise_std_normalization = True
    )

val_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


test_datagen = ImageDataGenerator(
    samplewise_center = True,
    samplewise_std_normalization = True
    )


In [ ]:
# Caminhos de treino e validação

# Obs - no diretório dados4 os dados estão separados em 75% treino 15% validação e 10% teste
path_train = '/content/drive/MyDrive/PG/dados4/train/'
path_val = '/content/drive/MyDrive/PG/dados4/val/'

batch_size = 20


In [ ]:
# dados de treino
train_generator = train_datagen.flow_from_directory(
    path_train,
    target_size = (299, 299),
    batch_size= batch_size,
    shuffle=True,
    class_mode='categorical' 
)

#dados de validação
val_generator = val_datagen.flow_from_directory(
    path_val,
    target_size = (299, 299),
    batch_size= batch_size,  
    shuffle=True,
    class_mode='categorical'     
)

Found 300 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


In [ ]:
y_true = train_generator.classes

In [ ]:
val_generator.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [ ]:
# Chamando a rede pré-treinada
conv_model = VGG19(
    weights='imagenet', 
    include_top=True, 
    )

conv_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# Chamando a rede pré-treinada
conv_model = VGG19(
    weights='imagenet', 
    include_top=False, 
    input_shape=(299,299,3))

first_freeze_layers = 20 #number of the first layers to freeze
for layer in conv_model.layers[0:first_freeze_layers]:
    layer.trainable = False

teste = keras.layers.Flatten()(conv_model.output)

# three hidden layers
teste = keras.layers.Dropout(0.4)(teste)
teste = keras.layers.Dense(128, activation='relu')(teste)
teste = keras.layers.Dense(64, activation='relu')(teste)

# final softmax layer with four categories
predictions = keras.layers.Dense(4, activation='softmax')(teste)

# creating the full model:
full_model = keras.models.Model(inputs=conv_model.input, outputs=predictions)

full_model.summary()

80150528/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 299, 299, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 299, 299, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 299, 299, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 149, 149, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 149, 149, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 149, 149, 128)     147584    
                                                              

In [ ]:
#EARLY STOPPING ADICIONADO
# https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
callback = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=100,
    verbose=0,
    mode="min",
    baseline=None,
    restore_best_weights=True
)

lr_schedule = keras.optimizers.schedules.CosineDecayRestarts(
    initial_learning_rate = 0.001, 
    first_decay_steps = 500, 
    t_mul=2.4, 
    m_mul=1.3, 
    alpha=0.5, 
    name=None
)

In [ ]:
# Compilando o modelo
full_model.compile(loss='categorical_crossentropy',
                  optimizer= keras.optimizers.SGD(learning_rate=lr_schedule),
                  metrics=['accuracy'])

# Treinando o modelo com os dados de treino e teste
history = full_model.fit(
    train_generator,
    validation_data = val_generator,
    epochs = 500,
    callbacks=[callback]
)

Epoch 1/500
15/15 [==============================] - 86s 5s/step - loss: 1.4804 - accuracy: 0.2533 - val_loss: 1.3329 - val_accuracy: 0.4000
Epoch 2/500
15/15 [==============================] - 10s 678ms/step - loss: 1.3882 - accuracy: 0.3167 - val_loss: 1.3018 - val_accuracy: 0.3500
Epoch 3/500
15/15 [==============================] - 10s 679ms/step - loss: 1.3692 - accuracy: 0.3067 - val_loss: 1.2963 - val_accuracy: 0.4167
Epoch 4/500
15/15 [==============================] - 10s 673ms/step - loss: 1.3408 - accuracy: 0.3933 - val_loss: 1.3001 - val_accuracy: 0.3167
Epoch 5/500
15/15 [==============================] - 10s 684ms/step - loss: 1.3079 - accuracy: 0.4067 - val_loss: 1.2231 - val_accuracy: 0.5000
Epoch 6/500
15/15 [==============================] - 11s 704ms/step - loss: 1.2636 - accuracy: 0.4600 - val_loss: 1.2578 - val_accuracy: 0.3833
Epoch 7/500
15/15 [==============================] - 11s 685ms/step - loss: 1.2916 - accuracy: 0.3767 - val_loss: 1.2121 - val_accuracy: 0.

In [ ]:
#dados de teste
path_test = '/content/drive/MyDrive/PG/dados4/test/'
test_generator =  test_datagen.flow_from_directory(
    path_test,
    target_size = (299, 299),
    shuffle = False,
    batch_size= batch_size,
    class_mode='categorical'     
    )

Found 40 images belonging to 4 classes.


In [ ]:
#Confution Matrix and Classification Report

from sklearn.metrics import confusion_matrix, classification_report
Y_pred = full_model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

cmvgg = confusion_matrix(test_generator.classes, y_pred )

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix(test_generator.classes, y_pred )

array([[ 7,  3,  0,  0],
       [ 1,  7,  2,  0],
       [ 1,  0,  8,  1],
       [ 0,  0,  0, 10]])

In [ ]:
print(classification_report(test_generator.classes, y_pred))


              precision    recall  f1-score   support

           0       0.78      0.70      0.74        10
           1       0.70      0.70      0.70        10
           2       0.80      0.80      0.80        10
           3       0.91      1.00      0.95        10

    accuracy                           0.80        40
   macro avg       0.80      0.80      0.80        40
weighted avg       0.80      0.80      0.80        40



In [ ]:
teste_generator.classes